In [100]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import dotenv_values
import requests
import plotly.express as px 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [2]:
# recuperando credencials em arquivo env por questão de privacidade
config = dotenv_values("credentials.env")

# setando credenciais
SPOTIPY_CLIENT_ID=config["SPOTIPY_CLIENT_ID"]
SPOTIPY_CLIENT_SECRET=config["SPOTIPY_CLIENT_SECRET"]
SPOTIPY_REDIRECT_URI=config["SPOTIPY_REDIRECT_URI"]

In [185]:
# inserindo credenciais na api e algumas chamadas especificas para endpoints que pedem um escopo especifico

# chamada geral para endpoints que não requerem um escopo
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = SPOTIPY_CLIENT_ID,
                                                      client_secret = SPOTIPY_CLIENT_SECRET,
                                                      redirect_uri = SPOTIPY_REDIRECT_URI
                                                      ))

# chamada para o endpoint de top items (artistas ou musicas)
sp_tops = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = SPOTIPY_CLIENT_ID,
                                                      client_secret = SPOTIPY_CLIENT_SECRET,
                                                      redirect_uri = SPOTIPY_REDIRECT_URI,
                                                      scope="user-top-read"
                                                      ))

    

##### Analisando a evolução de genero musical dos meus 50 artistas mais ouvidos

In [59]:
# função para gerar dataset de quantidade de artistas mais ouvidos por periodo de tempo
def top_artists(time_range, quant): 
    top_artists = sp_tops.current_user_top_artists(time_range=time_range, limit=quant)
    infos = []
    for item in top_artists["items"]:
        artista = {}
        artista["nome"] = item["name"]
        try:
            artista["genero_principal"] = item["genres"][0]
        except:
            artista["genero_principal"] = "Outros"
        try:
            artista["genero_secundario"] = item["genres"][1]
        except: 
            pass
        artista["id"] = item["id"]
        infos.append(artista)
    artists_df = pd.DataFrame()
    artists_df = artists_df.append(infos)

    generos = ["rock", "pop", "funk", "hip hop", "rap", "indie", "metal"]
    # encontra primeira palavra de uma lista de generos pré definidas por mim
    def checa_genero(x):
        for genero in generos:
            if genero in x:
                return genero
            else:
                pass 
    
    artists_df["genero_primario"] = artists_df["genero_principal"].apply(lambda x: checa_genero(x))
    artists_df["genero_primario"] = artists_df["genero_primario"].fillna("Outros")
    artists_df["time_range"] = time_range

    return artists_df

       

In [61]:
long_term = top_artists("long_term", 50)
medium_term = top_artists("medium_term", 50)

geral = pd.concat([long_term, medium_term])

In [71]:
fig = px.bar(geral, 
      x="genero_primario", 
      color="time_range",
      barmode="group", 
      hover_data=["nome", "genero_principal", "genero_secundario"],
      labels={"count": "Count", "genero_primario": "Genero Primario"},
      title="Generos dos meus 50 artistas mais ouvidos - Periodo longo x seis ultimos meses")
fig.update_layout(xaxis={"categoryorder": "total descending"})
fig.show()

##### Quais as features das minhas musicas mais ouvidas e sua evolução?


In [94]:
# features das minhas musicas mais ouvidas em um periodo de tempo
def top_tracks(time_range, quant): 
    top_tracks = sp_tops.current_user_top_tracks(time_range=time_range, limit=quant)
    ids = []
    for item in top_tracks["items"]:
        ids.append(item["id"])
    features = sp.audio_features(tracks=ids)
    tracks_df = pd.DataFrame()
    tracks_df = tracks_df.append(features)
    tracks_df["time_range"] = time_range

    return tracks_df

In [96]:
long_term_tracks = top_tracks("long_term", 50)
medium_term_tracks = top_tracks("medium_term", 50)

geral_tracks = pd.concat([long_term_tracks, medium_term_tracks])

In [147]:
""" colunas de interesse de acordo com a dscrição disponivel na documentação do spotify:
    - energy
    - instrumentalness
    - loudness
    - speechiness
"""

features = ["energy", "instrumentalness", "loudness", "speechiness"]
for feature in features: 
    fig = px.histogram(geral_tracks, x=feature, color="time_range", marginal="box")
    fig.update_traces(opacity=0.5)
    fig.show()

##### Criando uma playlist usando o algoritmo de recomendacão do Spotify com base nos generos musicais e features desejadas

In [ ]:
# criando uma playlist nova para inserir minhas recomendações
# chamada para o endpoint de modificações em playlists privadas
sp_play = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = SPOTIPY_CLIENT_ID,
                                                      client_secret = SPOTIPY_CLIENT_SECRET,
                                                      redirect_uri = SPOTIPY_REDIRECT_URI,
                                                      scope=["playlist-modify-public"]
                                                      ))

sp_play.user_playlist_create(user=sp.me()["id"], name="my recommendation algorithm", public=True, collaborative=False)

In [171]:
# parametros minimos das musicas que eu quero
min_energy = geral_tracks[geral_tracks["time_range"] == "medium_term"]["energy"].quantile([.50]).iloc[0]
min_loudness = geral_tracks[geral_tracks["time_range"] == "medium_term"]["loudness"].quantile([.50]).iloc[0]

In [218]:
rec = sp.recommendations(seed_genres=["rock", "punk-rock", "emo", "electro", "electronic"], limit=20, min_energy=min_energy, min_loudness=min_loudness, min_popularity=60)

In [220]:
lista_rec = []
for track in rec["tracks"]:
    lista_rec.append(track["uri"])

In [221]:
lista_rec

['spotify:track:07GvNcU1WdyZJq3XxP0kZa',
 'spotify:track:5Z5nbOXhsSbySVC7WUc6y9',
 'spotify:track:6gLn8QhAYL4dEwdpVjfCPl',
 'spotify:track:5MsZIaCYY6Tsdph0LiB0hE',
 'spotify:track:7kXmJwrZGIhDaLT9sNo3ut',
 'spotify:track:1p1nO35bbi4ZlQgjIA4oa4',
 'spotify:track:0DxN6Ywom8nnndyQXdSBPy',
 'spotify:track:2l57cfmCnOkwNX1tky02n1',
 'spotify:track:1KtD0xaLAikgIt5tPbteZQ',
 'spotify:track:4TIJ7zSBNejpoIPaWpWRKc',
 'spotify:track:16Cs9KsHzgunxaEfGrXysG',
 'spotify:track:1bhjMY5O0ZjB41OHcdRH0a',
 'spotify:track:6AbVJjzv7thIvmMCuhZrmK',
 'spotify:track:1EotcFbWTbMCXeKFVEtX6Y',
 'spotify:track:1yjY7rpaAQvKwpdUliHx0d',
 'spotify:track:2nLtzopw4rPReszdYBJU6h',
 'spotify:track:663Karu2rvKLdnY0eo1n3M',
 'spotify:track:2TfSHkHiFO4gRztVIkggkE',
 'spotify:track:5kr3j5Clb9rjEposoMyLVt',
 'spotify:track:40sl0jG01g4FZkCrBzQhZX']

In [238]:
sp_play.playlist_add_items(playlist_id="0iZ58FX6S0wOcFGc1GT4qK", items=lista_rec, position=0)

{'snapshot_id': 'NiwyNWRiYjg0ODk3ZTA3ZjM2MWJmZDUyMzUxNjcxZjFiZGViZjA2MTNl'}

##### Limpando a playlist para a próxima rodada de sugestões

In [235]:
# obtendo os dados de todos os itens na playlist
itens = []
for item in sp_play.playlist_items(playlist_id="0iZ58FX6S0wOcFGc1GT4qK")["items"]:
    itens.append(item["track"]["uri"])

In [237]:
sp_play.playlist_remove_all_occurrences_of_items(playlist_id="0iZ58FX6S0wOcFGc1GT4qK", items=itens)

{'snapshot_id': 'NSw1NDQwNGRhYTNhOWExOWI3NmJmMDA1ZDAyMjNjZmFhMWM0NjNmZDBi'}